In [29]:
# init
from pyquda.utils import gauge_utils
from pyquda.field import LatticeFermion
from pyquda import core, pyqcu, mpi
import os
import sys
from time import perf_counter
import cupy as cp
test_dir = os.path.dirname(os.path.abspath("./"))
sys.path.insert(0, os.path.join(test_dir, ".."))
os.environ["QUDA_RESOURCE_PATH"] = ".cache"
latt_size = [16, 16, 16, 32]
grid_size = [1, 1, 1, 1]
Lx, Ly, Lz, Lt = latt_size
Nd, Ns, Nc = 4, 4, 3
Gx, Gy, Gz, Gt = grid_size
latt_size = [Lx//Gx, Ly//Gy, Lz//Gz, Lt//Gt]
Lx, Ly, Lz, Lt = latt_size
Vol = Lx * Ly * Lz * Lt
mpi.init(grid_size)
device_tmps0 = LatticeFermion(latt_size, cp.zeros(
    (Lt, Lz, Ly, Lx, Ns, Nc), cp.complex128))
device_tmps1 = LatticeFermion(latt_size, cp.zeros(
    (Lt, Lz, Ly, Lx, Ns, Nc), cp.complex128))
latt_shape = (Lt, Lz, Ly, Lx//2, Ns, Nc)
param = pyqcu.QcuParam()
param.lattice_size = latt_size
mpi_dslash = core.getDslash(latt_size, -3.5, 0, 0, anti_periodic_t=False)
kappa = 0.125
U = gauge_utils.gaussGauge(latt_size, 0)
mpi_dslash.loadGauge(U)

Creating Gaussian distrbuted Lie group field with sigma = 1.000000e-01


In [30]:
# give ans first
ans_e = cp.random.random(latt_shape) + 1j * \
    cp.random.random(latt_shape)  # ans_e
ans_o = cp.random.random(latt_shape) + 1j * \
    cp.random.random(latt_shape)  # ans_o

In [31]:
# give x_o, b__o, r, r_tilde, p, v, s, t
x_o = cp.random.random(latt_shape) + 1j * cp.random.random(latt_shape)  # x_o
b_e = cp.zeros(latt_shape, cp.complex128)
b_o = cp.zeros(latt_shape, cp.complex128)
b__o = cp.zeros(latt_shape, cp.complex128)
r = cp.zeros(latt_shape, cp.complex128)
r_tilde = cp.zeros(latt_shape, cp.complex128)
p = cp.zeros(latt_shape, cp.complex128)
s = cp.zeros(latt_shape, cp.complex128)
v = cp.zeros(latt_shape, cp.complex128)
t = cp.zeros(latt_shape, cp.complex128)
# give r_norm2, MAX_ITER, TOL, rho_prev, rho, alpha, omega, beta, kappa
r_norm2 = 0
MAX_ITER = 1e3
TOL = 1e-16
rho_prev = 1
rho = 0
alpha = 1
omega = 1
beta = 0
kappa = 0.125

In [32]:
# define dslash
def dslash_eo(src_o):
    device_tmps1.data[1, :] = src_o
    pyqcu.dslashQcu(device_tmps0.even_ptr, device_tmps1.odd_ptr,
                    U.data_ptr, param, 0)  # D_eo
    return device_tmps0.data[0, :]


def dslash_oe(src_e):
    device_tmps1.data[0, :] = src_e
    pyqcu.dslashQcu(device_tmps0.odd_ptr, device_tmps1.even_ptr,
                    U.data_ptr, param, 1)  # D_oe
    return device_tmps0.data[1, :]


def dslash(src_o):
    return src_o-kappa**2*dslash_oe(dslash_eo(src_o))

In [33]:
# give b'_o(b__0)
b__o = (ans_o-kappa*dslash_oe(ans_e))+kappa*dslash_oe((ans_e-kappa*dslash_eo(ans_o)))

In [34]:
def dot(a, b):
    cp.cuda.runtime.deviceSynchronize()
    return cp.inner(a.flatten().conjugate(), b.flatten())


def diff(a, b):
    cp.cuda.runtime.deviceSynchronize()
    return cp.linalg.norm(a - b) / cp.linalg.norm(b)

In [35]:
# bistabcg
r = b__o - dslash(x_o)
r_tilde = r

In [36]:
t0 = perf_counter()
for i in range(1, int(MAX_ITER)):
    # print("## rho:", rho)
    rho = dot(r_tilde, r)
    # print("## beta:", beta)
    beta = (rho / rho_prev) * (alpha / omega)
    p = r + (p - v * omega) * beta
    # v = A * p
    v = dslash(p)
    # print("## alpha:", alpha)
    alpha = rho / dot(r_tilde, v)
    s = r - v * alpha
    # t = A * s
    t = dslash(s)
    # print("## omega:", omega)
    omega = dot(t, s) / dot(t, t)
    x_o = x_o + p * alpha + s * omega
    r = s - t * omega
    r_norm2 = cp.linalg.norm(r)
    print("##{}# r_norm2:{}".format(i, r_norm2))
    # break
    if (r_norm2 < TOL or i == MAX_ITER - 1):
        print("### turns:", i)
        break
    rho_prev = rho
t1 = perf_counter()

##1# r_norm2:52.703085999051744
##2# r_norm2:22.748362195219972
##3# r_norm2:81.4225348862321
##4# r_norm2:12.990724405224858
##5# r_norm2:6.382087127625847
##6# r_norm2:3.991893627972822
##7# r_norm2:3.2151827338831325
##8# r_norm2:1.3755490045624084
##9# r_norm2:1.256032874318324
##10# r_norm2:0.8502055810412771
##11# r_norm2:0.4382476263000956
##12# r_norm2:0.36157613901991204
##13# r_norm2:0.29218721172804396
##14# r_norm2:0.35994313938515454
##15# r_norm2:0.26157591118939144
##16# r_norm2:0.16149073942604783
##17# r_norm2:0.14461912857726056
##18# r_norm2:0.1441934360866909
##19# r_norm2:0.19311224928766005
##20# r_norm2:0.1818866517025627
##21# r_norm2:0.15981516371393253
##22# r_norm2:0.07420034433482453
##23# r_norm2:0.05739720342242751
##24# r_norm2:0.04031548934945521
##25# r_norm2:0.03567545144243681
##26# r_norm2:0.057771213521040754
##27# r_norm2:0.022700851478591907
##28# r_norm2:0.020276696195298153
##29# r_norm2:0.016917605412583157
##30# r_norm2:0.016625579973845415
##

In [37]:
print("### ans_o = ", ans_o[0, 0, 0, 0, :, :])
print("### x_o = ", x_o[0, 0, 0, 0, :, :])
print('## difference: ', diff(x_o, ans_o))
print(f'Qcu time: {t1 - t0} sec')

### ans_o =  [[0.3424012 +0.3532274j  0.36839124+0.21647012j 0.11741085+0.82981132j]
 [0.82698589+0.99077119j 0.36590288+0.25654971j 0.69557489+0.10385907j]
 [0.7083333 +0.28584504j 0.45771385+0.59800529j 0.82383404+0.98715991j]
 [0.22063985+0.5950945j  0.53853839+0.08843996j 0.95424866+0.9103208j ]]
### x_o =  [[0.3424012 +0.3532274j  0.36839124+0.21647012j 0.11741085+0.82981132j]
 [0.82698589+0.99077119j 0.36590288+0.25654971j 0.69557489+0.10385907j]
 [0.7083333 +0.28584504j 0.45771385+0.59800529j 0.82383404+0.98715991j]
 [0.22063985+0.5950945j  0.53853839+0.08843996j 0.95424866+0.9103208j ]]
## difference:  8.782607159092754e-16
Qcu time: 5.534150187999785 sec


In [38]:
x_e = ans_e-kappa*dslash_eo(ans_o)+kappa * \
    dslash_eo(x_o)  # equal to x_o == ans_o
print(diff(x_e, ans_e))

4.987696310498938e-16
